# Using Scalar Values as Constants in PyProforma

This notebook demonstrates how to use scalar values for line items that should have the same value across all years, such as:
- Tax rates
- Inflation assumptions
- Interest rates
- Other financial constants

**Key Concept**: Instead of defining a dict with the same value for each year, you can simply pass a scalar value (int or float) to `LineItem.values`.

## Setup

First, let's import the necessary components and define our model years.

In [1]:
from pyproforma import LineItem, Model

# Define the years for our model
years = [2024, 2025, 2026, 2027, 2028]

## Defining Constants with Scalar Values

Instead of passing a dictionary with the same value repeated for each year, we can pass a single scalar value. This is much more concise and clearer in intent.

In [2]:
# Tax rate - just pass a scalar value instead of a dict
tax_rate = LineItem(
    name="tax_rate",
    label="Corporate Tax Rate",
    values=0.21,  # Scalar value - same for all years
)

# Annual inflation assumption
inflation_rate = LineItem(
    name="inflation",
    label="Annual Inflation Rate",
    values=0.03,  # Scalar value
)

## Revenue and Operating Data

For values that change year-by-year, we still use the traditional dictionary approach.

In [3]:
# Revenue - different values per year
revenue = LineItem(
    name="revenue",
    label="Revenue",
    values={
        2024: 1000000,
        2025: 1100000,
        2026: 1210000,
        2027: 1331000,
        2028: 1464100,
    },
)

# Operating expenses - different values per year
opex = LineItem(
    name="opex",
    label="Operating Expenses",
    values={2024: 300000, 2025: 315000, 2026: 330750, 2027: 347287, 2028: 364652},
)

## Calculations Using Scalar Constants

Now we can use our scalar constants in formulas. The scalar values will automatically be applied to all years.

In [4]:
# EBIT (Earnings Before Interest and Tax)
ebit = LineItem(
    name="ebit",
    label="EBIT",
    formula="revenue - opex",
)

# Tax expense - uses the scalar tax_rate constant
tax_expense = LineItem(
    name="tax_expense",
    label="Tax Expense",
    formula="ebit * tax_rate",  # tax_rate is a scalar, same for all years
)

# Net Income
net_income = LineItem(
    name="net_income",
    label="Net Income",
    formula="ebit - tax_expense",
)

## Create the Model

Now let's create our model with all the line items.

In [5]:
model = Model(
    line_items=[
        tax_rate,
        inflation_rate,
        revenue,
        opex,
        ebit,
        tax_expense,
        net_income,
    ],
    years=years,
)

## Verify Scalar Constants

Let's verify that our scalar constants have the same value across all years.

In [6]:
print("SCALAR CONSTANTS (Same value across all years):")
print("-" * 60)
print(f"Tax Rate:        {model.value('tax_rate', 2024):.1%}")
print(f"Inflation:       {model.value('inflation', 2024):.1%}")
print()

# Verify scalars are consistent across all years
print("Verifying scalars are consistent:")
print("-" * 60)
for year in years:
    assert model.value("tax_rate", year) == 0.21
    assert model.value("inflation", year) == 0.03
print("✓ All scalars verified to be consistent across all years")

SCALAR CONSTANTS (Same value across all years):
------------------------------------------------------------
Tax Rate:        21.0%
Inflation:       3.0%

Verifying scalars are consistent:
------------------------------------------------------------
✓ All scalars verified to be consistent across all years


## View Calculations

Let's see how the scalar constants are used in our calculations across all years.

In [7]:
print("CALCULATIONS USING SCALAR CONSTANTS:")
print("=" * 60)
for year in years:
    revenue_val = model.value("revenue", year)
    ebit_val = model.value("ebit", year)
    tax = model.value("tax_expense", year)
    net = model.value("net_income", year)

    print(f"\n{year}:")
    print(f"  Revenue:           ${revenue_val:,.0f}")
    print(f"  EBIT:              ${ebit_val:,.0f}")
    print(f"  Tax (21%):         ${tax:,.0f}")
    print(f"  Net Income:        ${net:,.0f}")

CALCULATIONS USING SCALAR CONSTANTS:

2024:
  Revenue:           $1,000,000
  EBIT:              $700,000
  Tax (21%):         $147,000
  Net Income:        $553,000

2025:
  Revenue:           $1,100,000
  EBIT:              $785,000
  Tax (21%):         $164,850
  Net Income:        $620,150

2026:
  Revenue:           $1,210,000
  EBIT:              $879,250
  Tax (21%):         $184,642
  Net Income:        $694,608

2027:
  Revenue:           $1,331,000
  EBIT:              $983,713
  Tax (21%):         $206,580
  Net Income:        $777,133

2028:
  Revenue:           $1,464,100
  EBIT:              $1,099,448
  Tax (21%):         $230,884
  Net Income:        $868,564


## Generate a Table View

Let's create a nice table view of our model results.

In [8]:
from pyproforma import Table

table = Table(
    model=model,
    line_items=['revenue', 'opex', 'ebit', 'tax_rate', 'tax_expense', 'net_income'],
)

table

ImportError: cannot import name 'Table' from 'pyproforma' (C:\projects\pyproforma\pyproforma\pyproforma\__init__.py)

## Key Takeaways

1. **Use scalar values** (e.g., `values=0.21`) for constants that should be the same across all years
2. **Use dict values** (e.g., `values={2024: 100, 2025: 110}`) for year-specific values
3. **Scalars automatically apply** to all years in the model - no need to repeat the value
4. **Clearer intent**: Using a scalar makes it obvious that the value is meant to be constant
5. **Less error-prone**: No risk of accidentally setting different values for different years when you meant them to be the same

This feature is particularly useful for modeling assumptions like tax rates, interest rates, inflation rates, and other constants that don't change year-over-year in your model.